In [ ]:
!pip install --upgrade pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio


In [ ]:
import pandas as pd
import os
from datasets import Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torchaudio


In [ ]:
import pandas as pd
import os
from datasets import Dataset
import torchaudio

# --------- 1. Charger les CSV ---------
# Définir les chemins des fichiers audio
path_train = "/kaggle/input/common-voice/cv-other-train"
path_valid = "/kaggle/input/common-voice/cv-valid-test"

# Lire les CSVs et ne garder que les colonnes utiles
train_df = pd.read_csv("/kaggle/input/common-voice/cv-other-train.csv")[["filename", "text"]]
valid_df = pd.read_csv("/kaggle/input/common-voice/cv-valid-test.csv")[["filename", "text"]]

# Ajouter les chemins complets vers les fichiers audio
train_df["audio"] = train_df["filename"].apply(lambda x: os.path.join(path_train, x))
valid_df["audio"] = valid_df["filename"].apply(lambda x: os.path.join(path_valid, x))

# Supprimer l'ancienne colonne filename
train_df = train_df.drop(columns=["filename"])
valid_df = valid_df.drop(columns=["filename"])

# --------- 2. Convertir en HuggingFace Dataset ---------
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

# --------- 3. Sélectionner seulement 15k exemples ---------
train_dataset = train_dataset.select(range(10000))  # sélectionne les 15 000 premiers


In [ ]:
len(train_dataset)

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", task="transcribe")


In [ ]:
def prepare_example(example):
    speech_array, sr = torchaudio.load(example["audio"])
    resampler = torchaudio.transforms.Resample(sr, 16000)
    example["input_features"] = processor(resampler(speech_array).squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_features[0]
    example["labels"] = processor.tokenizer(example["text"], return_tensors="pt").input_ids[0]
    return example


In [ ]:
train_dataset = train_dataset.map(prepare_example)
valid_dataset = valid_dataset.map(prepare_example)


In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")


In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="EN", task="transcribe")


In [ ]:
# Take a sample text from your Common Voice training dataset
input_str = train_dataset[0]["text"]

# Tokenize
labels = processor.tokenizer(input_str).input_ids

# Decode with and without special tokens
decoded_with_special = processor.tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = processor.tokenizer.decode(labels, skip_special_tokens=True)

# Print results
print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")


In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id
)





In [ ]:
import evaluate
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    return {"wer": wer_metric.compute(predictions=pred_str, references=label_str)}


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-cv-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    warmup_steps=500,
    max_steps=200,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=50,
    predict_with_generate=True,
    generation_max_length=128,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    report_to=["none"]
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.tokenizer,
)

trainer.train()


In [ ]:
trainer.save_model("./whisper-cv-finetuned")
processor.save_pretrained("./whisper-cv-finetuned")


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

processor = WhisperProcessor.from_pretrained("./whisper-cv-finetuned")
model = WhisperForConditionalGeneration.from_pretrained("./whisper-cv-finetuned")
model.eval()


In [7]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

# Load processor and model directly from HF Hub
processor = WhisperProcessor.from_pretrained("imen0/whisper-cv-finetuned")
model = WhisperForConditionalGeneration.from_pretrained("imen0/whisper-cv-finetuned")

model.eval()


2025-06-17 15:30:31.891890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750174231.915572    1053 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750174231.922925    1053 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 384)
      (layers): ModuleList(
        (0-3): 4 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=384, out_features=384, bias=False)
            (v_proj): Linear(in_features=384, out_features=384, bias=True)
            (q_proj): Linear(in_features=384, out_features=384, bias=True)
            (out_proj): Linear(in_features=384, out_features=384, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          

In [14]:

model.generation_config.forced_decoder_ids = None
def transcribe(audio_path):
    speech, sr = torchaudio.load(audio_path)
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        speech = resampler(speech)

    input_features = processor(speech.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_features.to(model.device)

    predicted_ids = model.generate(
        input_features,
        task="transcribe",
        language="en"
    )
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    return transcription


print(transcribe("/kaggle/input/common-voice/cv-valid-dev/cv-valid-dev/sample-000003.mp3"))

 I felt that everything I owned would be destroyed
